In [185]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import glob
import re
import time
import seaborn as sns
import xgboost as xgb
import csv
import pickle
color = sns.color_palette()

In [186]:
RAW_DATA_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/raw_data/'
DATA_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/'
TRAIN_DATA = RAW_DATA_PATH + 'train.csv'
TEST_DATA = RAW_DATA_PATH + 'test.csv'
MACRO_DATA = RAW_DATA_PATH + 'macro.csv'
MODELS_PATH = '/kaggle/dev/ashish/sberbank-russian-housing-market/models/'
SUBMISSIONS_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/submissions/'

In [176]:
# macro_df.loc[macro_df['child_on_acc_pre_school'] == '#!', 'child_on_acc_pre_school'] = 0

# def toFloat(x):
#     try:
#         return float(x.replace(',', ''))
#     except:
#         return x

#macro_df['child_on_acc_pre_school'] = list(map(lambda x: toFloat(x), macro_df['child_on_acc_pre_school'].values))
#print(macro_df['child_on_acc_pre_school'])

0       45,713
1       45,713
2       45,713
3       45,713
4       45,713
5       45,713
6       45,713
7       45,713
8       45,713
9       45,713
10      45,713
11      45,713
12      45,713
13      45,713
14      45,713
15      45,713
16      45,713
17      45,713
18      45,713
19      45,713
20      45,713
21      45,713
22      45,713
23      45,713
24      45,713
25      45,713
26      45,713
27      45,713
28      45,713
29      45,713
         ...  
2454       NaN
2455       NaN
2456       NaN
2457       NaN
2458       NaN
2459       NaN
2460       NaN
2461       NaN
2462       NaN
2463       NaN
2464       NaN
2465       NaN
2466       NaN
2467       NaN
2468       NaN
2469       NaN
2470       NaN
2471       NaN
2472       NaN
2473       NaN
2474       NaN
2475       NaN
2476       NaN
2477       NaN
2478       NaN
2479       NaN
2480       NaN
2481       NaN
2482       NaN
2483       NaN
Name: child_on_acc_pre_school, Length: 2484, dtype: object
0       45713.0
1       45

In [187]:
missing_df = macro_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(['missing_count'], ascending=[False])
missing_df

,column_name,missing_count
90,provision_retail_space_modern_sqm,1754
89,provision_retail_space_sqm,1753
81,modern_education_share,1389
82,old_education_build_share,1389
92,theaters_viewers_per_1000_cap,1023
87,hospital_beds_available_per_cap,1023
88,hospital_bed_occupancy_per_year,1023
28,grp_growth,1023
94,museum_visitis_per_100_cap,1023
96,population_reg_sports_share,1023


In [189]:
# Prep
train_df = pd.read_csv(TRAIN_DATA, parse_dates=['timestamp'])
macro_df = pd.read_csv(MACRO_DATA, parse_dates=['timestamp'])
test_df = pd.read_csv(TEST_DATA, parse_dates=['timestamp'])

print('Train data', train_df.shape)
print('Test data', test_df.shape)
print('Macro data', macro_df.shape)

# Fix child_on_acc_pre_school column
# macro_df.loc[macro_df['child_on_acc_pre_school'] == '#!', 'child_on_acc_pre_school'] = 0 

print('Merging with macro data..')
# Merge train/test data with macro data
train_macro_df =  pd.merge(train_df, macro_df, left_on='timestamp', right_on='timestamp', how='inner')
assert(len(train_macro_df) == len(train_df))
test_macro_df =  pd.merge(test_df, macro_df, left_on='timestamp', right_on='timestamp', how='inner')
assert(len(test_macro_df) == len(test_df))

print('Train + Macro data', train_macro_df.shape)
print('Test + Macro data', test_macro_df.shape)

# Find incompatible data types between train and test data
# for i in train_macro_df.columns:
#     try:
#         if not(train_macro_df[i].dtype == test_macro_df[i].dtype):
#             print('train:', i, '-', train_macro_df[i].dtype, '; test:', i, '-', test_macro_df[i].dtype)
#     except KeyError:
#         print('KeyError- skipping for:', i)

print('Fixing dtypes..')
# Fix incompatible data types between train and test data
train_macro_df['full_sq'] = train_macro_df['full_sq'].astype('float64')
test_macro_df['floor'] = test_macro_df['floor'].astype('float64')
test_macro_df['max_floor'] = test_macro_df['max_floor'].astype('float64')
test_macro_df['material'] = test_macro_df['material'].astype('float64')
test_macro_df['num_room'] = test_macro_df['num_room'].astype('float64')

train_columns = set(train_macro_df.columns)
drop_columns = ['id', 'timestamp', 'price_doc', 'child_on_acc_pre_school']

print('Encoding columns..')
for col in drop_columns:
    if col in train_columns:
        train_columns.remove(col)

for f in train_columns:
    if train_macro_df[f].dtype=='object':
        # print('encoding column:', f, '(original dtype:', train_macro_df[f].dtype, ')')
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(set(train_macro_df[f].values) | set(test_macro_df[f].values)))
        try:
            train_macro_df[f] = lbl.transform(list(train_macro_df[f].values))
            test_macro_df[f] = lbl.transform(list(test_macro_df[f].values))
        except TypeError as err:
            print('Error encoding column: ', f, err)

def rmsle(y_predicted, y_true):
    y_pred = y_predicted
    y_label = y_true.get_label()
    temp = np.square(np.log(y_pred + 1.0) - np.log(y_label + 1.0))
    error = np.sqrt(np.mean(temp))
    error_std = np.sqrt(np.std(temp))
    print('rmsle:', error, '; std:', error_std)
    return ("", error)

X = train_macro_df[list(train_columns)]
Y = train_macro_df.price_doc.values

train_X, val_X, train_Y, val_Y = model_selection.train_test_split(X, Y, train_size=0.7, random_state=42)

print('train_X.shape', train_X.shape)
print('train_Y.shape', train_Y.shape)
print('val_X.shape', val_X.shape)
print('val_Y.shape', val_Y.shape)

Train data (30471, 292)
Test data (7662, 291)
Macro data (2484, 100)
Merging with macro data..
Train + Macro data (30471, 391)
Test + Macro data (7662, 390)
Fixing dtypes..
Encoding columns..
train_X.shape (21329, 387)
train_Y.shape (21329,)
val_X.shape (9142, 387)
val_Y.shape (9142,)


In [191]:
model = xgb.XGBRegressor(max_depth = 10,
                        gamma=0.5,
                        objective="reg:linear",
                        n_estimators=10000,
                        learning_rate=0.005,
                        nthread=12,
                        subsample=0.8,
                        colsample_bytree=0.70,
                        colsample_bylevel=0.70,
                        seed=43,
                        silent=True)

model.fit(train_X, train_Y, eval_set=[(train_X, train_Y), (val_X, val_Y)], verbose=True, eval_metric=rmsle, early_stopping_rounds=50)

model_id = "model-" + str(int(time.time()))
pickle.dump(model, open(MODELS_PATH + model_id + ".xgb", "wb"))

print("Saved model ", model_id)

rmsle: 5.27085 ; std: 2.07324
rmsle: 5.28525 ; std: 2.37562
[0]	validation_0-:5.27085	validation_1-:5.28525
Multiple eval metrics have been passed: 'validation_1-' will be used for early stopping.

Will train until validation_1- hasn't improved in 50 rounds.
rmsle: 4.56804 ; std: 1.8475
rmsle: 4.57297 ; std: 1.99644
[1]	validation_0-:4.56804	validation_1-:4.57297
rmsle: 4.16477 ; std: 1.73842
rmsle: 4.1679 ; std: 1.85587
[2]	validation_0-:4.16477	validation_1-:4.1679
rmsle: 3.87699 ; std: 1.66112
rmsle: 3.88103 ; std: 1.76729
[3]	validation_0-:3.87699	validation_1-:3.88103
rmsle: 3.65673 ; std: 1.60042
rmsle: 3.66072 ; std: 1.69952
[4]	validation_0-:3.65673	validation_1-:3.66072
rmsle: 3.47683 ; std: 1.55312
rmsle: 3.48084 ; std: 1.6456
[5]	validation_0-:3.47683	validation_1-:3.48084
rmsle: 3.32616 ; std: 1.50976
rmsle: 3.33068 ; std: 1.59807
[6]	validation_0-:3.32616	validation_1-:3.33068
rmsle: 3.19611 ; std: 1.47237
rmsle: 3.20077 ; std: 1.55811
[7]	validation_0-:3.19611	validation_

rmsle: 1.16138 ; std: 0.700376
rmsle: 1.17952 ; std: 0.748867
[74]	validation_0-:1.16138	validation_1-:1.17952
rmsle: 1.15118 ; std: 0.694782
rmsle: 1.16955 ; std: 0.743601
[75]	validation_0-:1.15118	validation_1-:1.16955
rmsle: 1.14108 ; std: 0.689503
rmsle: 1.15967 ; std: 0.738638
[76]	validation_0-:1.14108	validation_1-:1.15967
rmsle: 1.13114 ; std: 0.68427
rmsle: 1.14998 ; std: 0.733891
[77]	validation_0-:1.13114	validation_1-:1.14998
rmsle: 1.12144 ; std: 0.678925
rmsle: 1.14051 ; std: 0.729009
[78]	validation_0-:1.12144	validation_1-:1.14051
rmsle: 1.1119 ; std: 0.673735
rmsle: 1.13122 ; std: 0.724359
[79]	validation_0-:1.1119	validation_1-:1.13122
rmsle: 1.10251 ; std: 0.668628
rmsle: 1.12203 ; std: 0.719657
[80]	validation_0-:1.10251	validation_1-:1.12203
rmsle: 1.0933 ; std: 0.663505
rmsle: 1.11302 ; std: 0.715032
[81]	validation_0-:1.0933	validation_1-:1.11302
rmsle: 1.08425 ; std: 0.658456
rmsle: 1.10418 ; std: 0.710575
[82]	validation_0-:1.08425	validation_1-:1.10418
rmsle:

rmsle: 0.699022 ; std: 0.494327
rmsle: 0.735403 ; std: 0.617891
[147]	validation_0-:0.699022	validation_1-:0.735403
rmsle: 0.695234 ; std: 0.494097
rmsle: 0.731915 ; std: 0.618556
[148]	validation_0-:0.695234	validation_1-:0.731915
rmsle: 0.691535 ; std: 0.493968
rmsle: 0.728482 ; std: 0.619263
[149]	validation_0-:0.691535	validation_1-:0.728482
rmsle: 0.687837 ; std: 0.493994
rmsle: 0.725035 ; std: 0.619981
[150]	validation_0-:0.687837	validation_1-:0.725035
rmsle: 0.684236 ; std: 0.494014
rmsle: 0.721677 ; std: 0.620793
[151]	validation_0-:0.684236	validation_1-:0.721677
rmsle: 0.680624 ; std: 0.494086
rmsle: 0.718327 ; std: 0.621661
[152]	validation_0-:0.680624	validation_1-:0.718327
rmsle: 0.677052 ; std: 0.494288
rmsle: 0.715001 ; std: 0.622456
[153]	validation_0-:0.677052	validation_1-:0.715001
rmsle: 0.673521 ; std: 0.494482
rmsle: 0.711696 ; std: 0.623115
[154]	validation_0-:0.673521	validation_1-:0.711696
rmsle: 0.670044 ; std: 0.494567
rmsle: 0.708481 ; std: 0.623936
[155]	va

rmsle: 0.511873 ; std: 0.544564
rmsle: 0.567086 ; std: 0.692141
[219]	validation_0-:0.511873	validation_1-:0.567086
rmsle: 0.510195 ; std: 0.545484
rmsle: 0.565681 ; std: 0.693223
[220]	validation_0-:0.510195	validation_1-:0.565681
rmsle: 0.508529 ; std: 0.54635
rmsle: 0.564277 ; std: 0.694228
[221]	validation_0-:0.508529	validation_1-:0.564277
rmsle: 0.506867 ; std: 0.547214
rmsle: 0.562911 ; std: 0.695271
[222]	validation_0-:0.506867	validation_1-:0.562911
rmsle: 0.505252 ; std: 0.548106
rmsle: 0.561543 ; std: 0.69624
[223]	validation_0-:0.505252	validation_1-:0.561543
rmsle: 0.503635 ; std: 0.548959
rmsle: 0.560196 ; std: 0.697268
[224]	validation_0-:0.503635	validation_1-:0.560196
rmsle: 0.502025 ; std: 0.549844
rmsle: 0.558843 ; std: 0.698287
[225]	validation_0-:0.502025	validation_1-:0.558843
rmsle: 0.500463 ; std: 0.550702
rmsle: 0.55753 ; std: 0.699239
[226]	validation_0-:0.500463	validation_1-:0.55753
rmsle: 0.49887 ; std: 0.551608
rmsle: 0.556197 ; std: 0.700281
[227]	validat

rmsle: 0.428167 ; std: 0.599027
rmsle: 0.499612 ; std: 0.753957
[290]	validation_0-:0.428167	validation_1-:0.499612
rmsle: 0.427416 ; std: 0.599757
rmsle: 0.499017 ; std: 0.754658
[291]	validation_0-:0.427416	validation_1-:0.499017
rmsle: 0.426637 ; std: 0.600323
rmsle: 0.498444 ; std: 0.755338
[292]	validation_0-:0.426637	validation_1-:0.498444
rmsle: 0.425822 ; std: 0.600762
rmsle: 0.497883 ; std: 0.756001
[293]	validation_0-:0.425822	validation_1-:0.497883
rmsle: 0.425079 ; std: 0.601393
rmsle: 0.497306 ; std: 0.756691
[294]	validation_0-:0.425079	validation_1-:0.497306
rmsle: 0.42435 ; std: 0.601987
rmsle: 0.496759 ; std: 0.757376
[295]	validation_0-:0.42435	validation_1-:0.496759
rmsle: 0.423636 ; std: 0.602592
rmsle: 0.496215 ; std: 0.75804
[296]	validation_0-:0.423636	validation_1-:0.496215
rmsle: 0.422885 ; std: 0.603113
rmsle: 0.495681 ; std: 0.758714
[297]	validation_0-:0.422885	validation_1-:0.495681
rmsle: 0.422159 ; std: 0.603636
rmsle: 0.495164 ; std: 0.759376
[298]	valid

rmsle: 0.389173 ; std: 0.631781
rmsle: 0.472919 ; std: 0.795531
[362]	validation_0-:0.389173	validation_1-:0.472919
rmsle: 0.388791 ; std: 0.632073
rmsle: 0.472712 ; std: 0.795992
[363]	validation_0-:0.388791	validation_1-:0.472712
rmsle: 0.388397 ; std: 0.632274
rmsle: 0.472514 ; std: 0.796462
[364]	validation_0-:0.388397	validation_1-:0.472514
rmsle: 0.388028 ; std: 0.632568
rmsle: 0.472304 ; std: 0.796916
[365]	validation_0-:0.388028	validation_1-:0.472304
rmsle: 0.387668 ; std: 0.632884
rmsle: 0.47207 ; std: 0.797345
[366]	validation_0-:0.387668	validation_1-:0.47207
rmsle: 0.387279 ; std: 0.633101
rmsle: 0.471861 ; std: 0.797781
[367]	validation_0-:0.387279	validation_1-:0.471861
rmsle: 0.386978 ; std: 0.633503
rmsle: 0.471659 ; std: 0.798205
[368]	validation_0-:0.386978	validation_1-:0.471659
rmsle: 0.386645 ; std: 0.633819
rmsle: 0.471471 ; std: 0.798668
[369]	validation_0-:0.386645	validation_1-:0.471471
rmsle: 0.386365 ; std: 0.63423
rmsle: 0.471279 ; std: 0.799088
[370]	valid

rmsle: 0.37077 ; std: 0.649202
rmsle: 0.463638 ; std: 0.822606
[434]	validation_0-:0.37077	validation_1-:0.463638
rmsle: 0.370552 ; std: 0.649277
rmsle: 0.463567 ; std: 0.822902
[435]	validation_0-:0.370552	validation_1-:0.463567
rmsle: 0.370365 ; std: 0.649411
rmsle: 0.463499 ; std: 0.823189
[436]	validation_0-:0.370365	validation_1-:0.463499
rmsle: 0.37018 ; std: 0.649546
rmsle: 0.46344 ; std: 0.823491
[437]	validation_0-:0.37018	validation_1-:0.46344
rmsle: 0.370008 ; std: 0.649686
rmsle: 0.463385 ; std: 0.823791
[438]	validation_0-:0.370008	validation_1-:0.463385
rmsle: 0.369873 ; std: 0.649926
rmsle: 0.46333 ; std: 0.824125
[439]	validation_0-:0.369873	validation_1-:0.46333
rmsle: 0.369697 ; std: 0.65006
rmsle: 0.463275 ; std: 0.824432
[440]	validation_0-:0.369697	validation_1-:0.463275
rmsle: 0.369532 ; std: 0.650224
rmsle: 0.463209 ; std: 0.824722
[441]	validation_0-:0.369532	validation_1-:0.463209
rmsle: 0.369349 ; std: 0.650328
rmsle: 0.463155 ; std: 0.825038
[442]	validation_

rmsle: 0.360871 ; std: 0.656768
rmsle: 0.461124 ; std: 0.840231
[506]	validation_0-:0.360871	validation_1-:0.461124
rmsle: 0.36078 ; std: 0.656873
rmsle: 0.461124 ; std: 0.840434
[507]	validation_0-:0.36078	validation_1-:0.461124
rmsle: 0.360643 ; std: 0.656871
rmsle: 0.461127 ; std: 0.84066
[508]	validation_0-:0.360643	validation_1-:0.461127
rmsle: 0.360551 ; std: 0.656954
rmsle: 0.461119 ; std: 0.840863
[509]	validation_0-:0.360551	validation_1-:0.461119
rmsle: 0.360506 ; std: 0.657135
rmsle: 0.461113 ; std: 0.841074
[510]	validation_0-:0.360506	validation_1-:0.461113
rmsle: 0.360381 ; std: 0.657154
rmsle: 0.461098 ; std: 0.841278
[511]	validation_0-:0.360381	validation_1-:0.461098
rmsle: 0.360262 ; std: 0.657182
rmsle: 0.461088 ; std: 0.841485
[512]	validation_0-:0.360262	validation_1-:0.461088
rmsle: 0.360153 ; std: 0.657233
rmsle: 0.461079 ; std: 0.841672
[513]	validation_0-:0.360153	validation_1-:0.461079
rmsle: 0.360072 ; std: 0.657308
rmsle: 0.461068 ; std: 0.841879
[514]	valid

rmsle: 0.355063 ; std: 0.659792
rmsle: 0.461129 ; std: 0.852082
[577]	validation_0-:0.355063	validation_1-:0.461129
rmsle: 0.354973 ; std: 0.659775
rmsle: 0.461138 ; std: 0.852222
[578]	validation_0-:0.354973	validation_1-:0.461138
rmsle: 0.35486 ; std: 0.659697
rmsle: 0.461145 ; std: 0.85235
[579]	validation_0-:0.35486	validation_1-:0.461145
rmsle: 0.354839 ; std: 0.659809
rmsle: 0.46115 ; std: 0.852477
[580]	validation_0-:0.354839	validation_1-:0.46115
rmsle: 0.354757 ; std: 0.65981
rmsle: 0.461143 ; std: 0.852583
[581]	validation_0-:0.354757	validation_1-:0.461143
rmsle: 0.354699 ; std: 0.659828
rmsle: 0.461144 ; std: 0.852689
[582]	validation_0-:0.354699	validation_1-:0.461144
rmsle: 0.35463 ; std: 0.659813
rmsle: 0.461152 ; std: 0.852811
[583]	validation_0-:0.35463	validation_1-:0.461152
rmsle: 0.35459 ; std: 0.659866
rmsle: 0.461161 ; std: 0.852946
[584]	validation_0-:0.35459	validation_1-:0.461161
rmsle: 0.354553 ; std: 0.659927
rmsle: 0.461171 ; std: 0.853081
[585]	validation_0

In [200]:
test_columns = list(train_columns)
test_columns.append('id')

test_X =  test_macro_df[test_columns]
print("test_X.shape", test_X.shape)

test_X.shape (7662, 388)


In [202]:
submission_columns = ['id', 'predicted_price_doc']
model = pickle.load(open(MODELS_PATH + model_id + ".xgb", 'rb'))
filename = SUBMISSIONS_PATH + 'submission-' + str(int(time.time())) + ".csv"
test_X.loc[:, 'predicted_price_doc'] = pd.Series(model.predict(test_X.loc[:, train_columns]), index=test_X.index)
predicted_submission_df = test_X[submission_columns]
predicted_submission_df.columns = ['id', 'price_doc']
predicted_submission_df.to_csv(filename, index=False)
print('Generated submission file: {}'.format(filename))

Generated submission file: /kaggle/dev/sberbank-russian-housing-market-data/submissions/submission-1495428713.csv


/kaggle/dev/ashish/sberbank-russian-housing-market/sber/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0        5351186.50
1        8143073.00
2        5849541.50
3        5850479.50
4        4958602.00
5        8844848.00
6        4391357.00
7        4225402.00
8        4393089.50
9        4542285.00
10       6556579.00
11       4868172.00
12       3819550.25
13       4381976.50
14       6009517.50
15       5574314.00
16      22028638.00
17      19201786.00
18       6084290.00
19      15598946.00
20       6973422.50
21      10406794.00
22       8174750.00
23       7635058.50
24       4668190.00
25       6849025.50
26      11282550.00
27       6909426.00
28       3217808.50
29       6434539.00
           ...     
7632     5955536.50
7633     3364783.50
7634    10680146.00
7635     5166003.00
7636     4448744.50
7637     4799900.00
7638     3094004.75
7639     7393709.00
7640     6699045.00
7641     2970651.00
7642     7031517.50
7643     6214014.00
7644     7866112.00
7645     3293062.25
7646     5695784.50
7647     6721641.00
7648     7105436.50
7649     6072870.50
7650     6633327.50
